## 0. Import Libraries

In [1]:
import re 
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

## 1.Scrape content off lyrics.com

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url ='https://www.lyrics.com'
artist_url = 'https://www.lyrics.com/artist.php?name=Elton-John&aid=4617&o=1'
artist2_url = 'https://www.lyrics.com/artist.php?name=The-Beatles&aid=3644&o=1'


In [3]:
# define variables for code
no_of_songs = 400

In [4]:
container = requests.get(artist_url, headers)
container2= requests.get(artist2_url, headers)

In [5]:
container_html = soup(container.text, 'html.parser')
container2_html = soup(container2.text, 'html.parser')


In [6]:
link = container_html.find_all('a', href=True)
link2 = container2_html.find_all('a', href=True)


In [7]:
all_links = [x.get('href') for x in link if r'/lyric' in x.get('href')]
all_links2 = [x.get('href') for x in link2 if r'/lyric' in x.get('href')]


In [8]:
df = pd.DataFrame()
#save all songs in one csv
for a in all_links[:no_of_songs]:
    # open required url
    response = requests.get(url+a, headers)
    # get html text from site
    response_html = soup(response.text, 'html.parser')
    # identify the song text
    response_text=response_html.find_all('pre',attrs={"id":"lyric-body-text"})
    # identify the song title
    response_title = response_html.find('h1',attrs={"class":"lyric-title"}).text
    # loop over song text and extract pure text / get rid of html technical things
    for i in response_text:
        song = i.text
        df = df.append([song],ignore_index=True)


In [9]:
df.to_csv('elton.csv')

In [10]:
df2 = pd.DataFrame()
#save all songs in one csv
for a in all_links2[:no_of_songs]:
    # open required url
    response2 = requests.get(url+a, headers)
    # get html text from site
    response2_html = soup(response2.text, 'html.parser')
    # identify the song text
    response2_text=response2_html.find_all('pre',attrs={"id":"lyric-body-text"})
    # identify the song title
    response2_title = response2_html.find('h1',attrs={"class":"lyric-title"}).text
    # loop over song text and extract pure text / get rid of html technical things
    for i in response2_text:
        song2 = i.text
        df2 = df2.append([song2],ignore_index=True)

In [11]:
df2.to_csv('beatles.csv')

In [12]:
artist_list = [["Elton John"] * no_of_songs + ["The Beatles"] * no_of_songs]
df3 = pd.concat([df, df2], ignore_index=True).set_index(artist_list).rename(columns={0:'songtext'})
df3['str_compare']=df3['songtext'].str.slice(0,20)
df3 = df3.drop_duplicates(subset=['str_compare'], keep='first')
df3 = df3.drop('str_compare', axis =1)
df3

,songtext
Elton John,I can hound you if I need to\nSip your brandy ...
Elton John,Sitting here on Christmas Eve with a brandy in...
Elton John,"Oh, the joke was never hard to tell\nThrew my ..."
Elton John,Oh it's great to see our homeland\nBreathe the...
Elton John,"Hold up, y'all\nHold up\nNot yet\nHold up\n\nN..."
...,...
The Beatles,"One, two, three, four\nCan I have a little mor..."
The Beatles,"Love, love, love\nLove, love, love\nLove, love..."
The Beatles,"Love, love, love.\r\nLove, love, love.\r\nLove..."
The Beatles,"Love, love, love\r\nLove, love, love\r\nLove, ..."


In [13]:
artist_new = df3.index
artist_new


Index(['Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'Elton John',
       'Elton John', 'Elton John', 'Elton John', 'Elton John', 'The Beatles'

In [14]:
df3.shape, artist_new.shape

((83, 1), (83,))

## 2. Apply Bag of Words

In [15]:
cv = CountVectorizer(stop_words='english')
cv.fit(df3['songtext'])

CountVectorizer(stop_words='english')

In [16]:
df3_vecs = cv.transform(df3['songtext'])
df3_vecs.todense() 

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [17]:
df3_cv = pd.DataFrame(df3_vecs.todense(), index = artist_new, columns=cv.get_feature_names())
df3_cv

,abide,able,abroad,accent,act,acting,action,acts,ad,addict,...,yeah,year,years,yee,yellow,yes,yesterday,yo,young,youth
Elton John,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elton John,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elton John,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Elton John,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elton John,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Beatles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
The Beatles,0,0,0,0,0,0,0,0,0,0,...,7,0,0,0,0,0,1,0,0,0
The Beatles,0,0,0,0,0,0,0,0,0,0,...,7,0,0,1,0,0,0,0,0,0
The Beatles,0,0,0,0,0,0,0,0,0,0,...,7,0,0,1,0,0,0,0,0,0


## 3. Apply TfIDf Transformer

In [18]:
tf = TfidfTransformer()
tf_vecs = tf.fit_transform(df3_vecs)
tf_vecs.shape

(83, 1944)

In [19]:
df3_tf = pd.DataFrame(tf_vecs.todense().round(2), index=artist_new, columns=cv.get_feature_names())
df3_tf

,abide,able,abroad,accent,act,acting,action,acts,ad,addict,...,yeah,year,years,yee,yellow,yes,yesterday,yo,young,youth
Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.04,0.00,0.00,0.0,0.00,0.00,0.0,0.0
Elton John,0.0,0.0,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.04,0.0,0.00,0.00,0.00,0.0,0.00,0.06,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.05,0.0,0.00,0.00,0.0,0.0
The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.11,0.0,0.00,0.00,0.00,0.0,0.02,0.00,0.0,0.0
The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0
The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0


## 3. Run Naive Bayes Model

In [20]:
df3_tf.reset_index(inplace=True)
#df3_tf.rename(columns={'level_0':'artist'}, inplace=True).drop('index')
df3_tf

,index,abide,able,abroad,accent,act,acting,action,acts,ad,...,yeah,year,years,yee,yellow,yes,yesterday,yo,young,youth
0,Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
1,Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
2,Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.04,0.00,0.00,0.0,0.00,0.00,0.0,0.0
3,Elton John,0.0,0.0,0.07,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
4,Elton John,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.04,0.0,0.00,0.00,0.00,0.0,0.00,0.06,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.05,0.0,0.00,0.00,0.0,0.0
79,The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.11,0.0,0.00,0.00,0.00,0.0,0.02,0.00,0.0,0.0
80,The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0
81,The Beatles,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0


In [21]:
y = df3_tf['index']
x = df3_tf.drop('index', axis =1)
x

,abide,able,abroad,accent,act,acting,action,acts,ad,addict,...,yeah,year,years,yee,yellow,yes,yesterday,yo,young,youth
0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
2,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.04,0.00,0.00,0.0,0.00,0.00,0.0,0.0
3,0.0,0.0,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0
4,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.04,0.0,0.00,0.00,0.00,0.0,0.00,0.06,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.00,0.05,0.0,0.00,0.00,0.0,0.0
79,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.11,0.0,0.00,0.00,0.00,0.0,0.02,0.00,0.0,0.0
80,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0
81,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24,0.0,0.00,0.05,0.00,0.0,0.00,0.00,0.0,0.0


In [22]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, random_state=87)

In [23]:
mn = MultinomialNB(alpha=1)

In [24]:
mn.fit(xtrain, ytrain)

MultinomialNB(alpha=1)

In [25]:
mn.score(xtest, ytest)

0.7142857142857143

In [26]:
y_train_pred = mn.predict(xtrain)
print('y_train_pred: ',y_train_pred)

y_train_pred:  ['Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John' 'Elton John' 'Elton John' 'Elton John'
 'Elton John' 'Elton John']


In [27]:
training_score = mn.score(xtrain, ytrain)
training_score

0.7903225806451613